In [1]:
import requests
import numpy as np
import matplotlib.pyplot as plt
import base64

In [2]:
#curl --verbose --request POST -d '{"top_k": 1, "mode": "search",  "data": ["data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAIAAABLbSncAAAA2ElEQVR4nADIADf/AxWcWRUeCEeBO68T3u1qLWarHqMaxDnxhAEaLh0Ssu6ZGfnKcjP4CeDLoJok3o4aOPYAJocsjktZfo4Z7Q/WR1UTgppAAdguAhR+AUm9AnqRH2jgdBZ0R+kKxAFoAME32BL7fwQbcLzhw+dXMmY9BS9K8EarXyWLH8VYK1MACkxlLTY4Eh69XfjpROqjE7P0AeBx6DGmA8/lRRlTCmPkL196pC0aWBkVs2wyjqb/LABVYL8Xgeomjl3VtEMxAeaUrGvnIawVh/oBAAD///GwU6v3yCoVAAAAAElFTkSuQmCC", "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAIAAABLbSncAAAA2ElEQVR4nADIADf/AvdGjTZeOlQq07xSYPgJjlWRwfWEBx2+CgAVrPrP+O5ghhOa+a0cocoWnaMJFAsBuCQCgiJOKDBcIQTiLieOrPD/cp/6iZ/Iu4HqAh5dGzggIQVJI3WqTxwVTDjs5XJOy38AlgHoaKgY+xJEXeFTyR7FOfF7JNWjs3b8evQE6B2dTDvQZx3n3Rz6rgOtVlaZRLvR9geCAxuY3G+0mepEAhrTISES3bwPWYYi48OUrQOc//IaJeij9xZGGmDIG9kc73fNI7eA8VMBAAD//0SxXMMT90UdAAAAAElFTkSuQmCC"]}' -H 'Content-Type: application/json' 'http://0.0.0.0:45678/api/search' > response.txt
url = 'http://0.0.0.0:45678/api/search'

js = {
    "top_k": 20, 
    "mode": "search",  
    "data": [
        "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAIAAABLbSncAAAA2ElEQVR4nADIADf/AxWcWRUeCEeBO68T3u1qLWarHqMaxDnxhAEaLh0Ssu6ZGfnKcjP4CeDLoJok3o4aOPYAJocsjktZfo4Z7Q/WR1UTgppAAdguAhR+AUm9AnqRH2jgdBZ0R+kKxAFoAME32BL7fwQbcLzhw+dXMmY9BS9K8EarXyWLH8VYK1MACkxlLTY4Eh69XfjpROqjE7P0AeBx6DGmA8/lRRlTCmPkL196pC0aWBkVs2wyjqb/LABVYL8Xgeomjl3VtEMxAeaUrGvnIawVh/oBAAD///GwU6v3yCoVAAAAAElFTkSuQmCC", "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAgAAAAICAIAAABLbSncAAAA2ElEQVR4nADIADf/AvdGjTZeOlQq07xSYPgJjlWRwfWEBx2+CgAVrPrP+O5ghhOa+a0cocoWnaMJFAsBuCQCgiJOKDBcIQTiLieOrPD/cp/6iZ/Iu4HqAh5dGzggIQVJI3WqTxwVTDjs5XJOy38AlgHoaKgY+xJEXeFTyR7FOfF7JNWjs3b8evQE6B2dTDvQZx3n3Rz6rgOtVlaZRLvR9geCAxuY3G+0mepEAhrTISES3bwPWYYi48OUrQOc//IaJeij9xZGGmDIG9kc73fNI7eA8VMBAAD//0SxXMMT90UdAAAAAElFTkSuQmCC"
    ]
}
response = requests.post(url=url, json=js)



ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=45678): Max retries exceeded with url: /api/search (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8064546d10>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
js_response = response.json()

In [ ]:


def arr_from_blob(arr_serialized):
    arr_ser_bin = base64.b64decode(arr_serialized['buffer'])
    arr = np.frombuffer(arr_ser_bin, dtype=arr_serialized['dtype']).reshape(arr_serialized['shape'])
    arr = arr.transpose(1,2,0).astype('int')
    return arr

In [ ]:
arr_serialized = js_response['search']['docs'][0]['blob']
arr = arr_from_blob(arr_serialized)
plt.imshow(arr)

In [ ]:
decoded = base64.b64decode(js['data'][0].split('data:image/png;base64,')[1], validate=True)
decoded[:10]

In [ ]:
with open("zaza.png", "wb") as f:
    f.write(decoded)

In [ ]:
with open("./image_0005.jpg", "rb") as f:
    im_q = f.read()
    
im_q_64 = base64.b64encode(im_q)

In [ ]:
im_q_64[:1000]

In [ ]:
js = {
    "top_k": 30, 
    "mode": "search",  
    "data": [
        "data:image/png;base64," + im_q_64.decode()
    ]
}
response = requests.post(url=url, json=js)

In [ ]:
js_response = response.json()

In [ ]:
arr_serialized = js_response['search']['docs'][0]['blob']
arr = arr_from_blob(arr_serialized)
plt.imshow(arr)

In [ ]:
from jina.flow import Flow
import os 

os.environ['PARALLEL'] = str(4)
os.environ['SHARDS'] = str(2)
os.environ['DATA_DIR'] = os.environ.get('DATA_DIR', './data/jpg')
os.environ['COLOR_CHANNEL_AXIS'] = str(0)
os.environ['JINA_PORT'] = str(45678)
os.environ['ENCODER'] = 'yaml/encode.yml'
os.environ['WORKDIR'] = os.environ.get('WORKDIR', './data/workdir')
os.environ['TMP_WORKSPACE'] = os.environ.get('TMP_WORKSPACE', './data/workdir')
f = Flow.load_config('./flow-query.yml')

In [ ]:
f.use_grpc_gateway()

In [ ]:
with f:
    f.search_files(image_src, sampling_rate=.01, batch_size=8, output_fn=print_result, top_k=5)

In [ ]:
# write_html('result.html')

In [ ]:
pip install torchvision